In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-kafka-struct-stream").\
        master("spark://spark:7077").\
        config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.postgresql:postgresql:42.5.1').\
        getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a8bf4a0-2577-4345-912d-ee865602d5c1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commo

23/01/20 15:36:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "test") \
    .load()

In [3]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructField, StructType, StringType

schema = StructType(
        [
                StructField("val1", StringType()),
                StructField("val2", StringType())
        ]
)

query = df.selectExpr("CAST(value AS STRING)") \
    .withColumn("value", from_json("value", schema))\
    .select(col('value.*')) \
    .writeStream \
    .format("console") \
    .option("checkpointLocation", "path/to/HDFS/dir") \
    .start()

query.awaitTermination()

23/01/20 15:36:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+
|val1|val2|
+----+----+
+----+----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+----------+
|     val1|      val2|
+---------+----------+
|aucbmdqbw|wnkzcgcvmq|
|   qzbeoq|tsofmjfjlv|
+---------+----------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------------+----------+
|         val1|      val2|
+-------------+----------+
| gfchwmzmgigh|fiimoocvvc|
|olffxwgbqwgic|     gezpz|
|   zwuklxyieb|kkfdwodcdi|
+-------------+----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------+------------+
|          val1|        val2|
+--------------+------------+
|  wtodegjcqrtc|txydlxhwfumt|
|kpwbhsejxjexaz| fhljofaqoih|
+--------------+------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+----------+------------+
|      val1|        val2|
+----------+------------+
|      veya|         hzh|
|umgcmjkqdn|dwggzzmtripe|
|     amdtq| tacemjoyqxt|
+----------+------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----+
|   val1| val2|
+-------+-----+
|   null| null|
|bfpyhxz|kuwpt|
+-------+-----+

-------------------------------------------
Batch: 6
-------------------------------------------
+-----------+--------------+
|       val1|          val2|
+-----------+--------------+
|      yixgv| uvdfddwftjlyu|
|iggvgjpstbi|fuayrvysstwaoo|
|   zk

...

-------------------------------------------
Batch: 42
-------------------------------------------
+----+--------+
|val1|    val2|
+----+--------+
|sule|bkglexml|
+----+--------+



-------------------------------------------
Batch: 43
-------------------------------------------
+--------------+-----------+
|          val1|       val2|
+--------------+-----------+
|zszitndtkhqoad|   pmjsqpvf|
| sqpdgtbzfzqzv|ucypitxcvmy|
|  dinfilcznijm|      xmdtv|
+--------------+-----------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


-------------------------------------------
Batch: 44
-------------------------------------------
+-------+---------+
|   val1|     val2|
+-------+---------+
| shjfcr|    mviyi|
|eygzuok|ysdusmcri|
+-------+---------+



KeyboardInterrupt: 